In [1]:
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
import nltk
import json
import requests
import pandas as pd

from os.path import exists
from os.path import isdir

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ucloud/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/ucloud/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
#model = BloomForCausalLM.from_pretrained("bigscience/bloom") #remove -1b7 for whole modelb
#tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom")

In [3]:
#prompt = "The train drove directly down in"
#result_length = 20
#inputs = tokenizer(prompt, return_tensors="pt") #??

name = 'bloom_API_validation2'

In [ ]:
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_length=result_length
                      )[0]))

# Load train and validation data

In [4]:
def error(msg):
    print('  [\033[91mx\033[0m] ' + msg)
    exit(1)

def success(msg):
    print('  [\033[92mo\033[0m] ' + msg)
    
def load_json_lines(f):
    if not exists(f):
        error('The file "' + f + '" does not exist.')

    ret = []
    num = 1
    
    if isdir(f):
        f = glob(f + '/*.json*')
        
        if len(f) != 1:
            error('The input is an directory that contains multiple json files. Please create only a single json file. Got ' + str(f))
        
        f = f[0]
    
    with open(f, 'r') as inp:
        for l in inp:
            try:
                ret += [json.loads(l)]
            except:
                error('Invalid line ' + str(num) + ' in "' + f + '" with content: ' + l.strip())
            num += 1

    success('The file ' + f + ' is in JSONL format.')
    return ret

In [5]:
train = load_json_lines('train.jsonl')
validation = load_json_lines('validation.jsonl')
test = load_json_lines('input.jsonl')

  [o] The file train.jsonl is in JSONL format.
  [o] The file validation.jsonl is in JSONL format.
  [o] The file input.jsonl is in JSONL format.


In [ ]:
validation[130]['targetDescription'][-3:]

In [ ]:
i = 2
validation[i]['targetTitle'], validation[i]['postText'], validation[i]['targetDescription'], validation[i]['targetKeywords']

In [ ]:
len(' '.join(validation[3]['targetParagraphs']))

In [ ]:
"abcdefghijkl"[-5:]

In [6]:
#Function for creating prompt from entry

def make_prompt(entry):
    article = ' '.join(entry['targetParagraphs'])
    title = entry['targetTitle']
    description = entry['targetDescription']
    tags = entry['targetKeywords']
    clickbait = entry['postText'][0]
    
    words = ['what', 'why']
    
    if len(article) > 4000:
        article = article[:2500] + article[-1000:]
    
    if title.lower() == clickbait.lower():
        title = ''
    else:
        title = f"\n\nArticle title: {title}"
    
    if tags:
        tags = f"\n\nTags: {', '.join(tags.split(','))}"
    else:
        tags = ''
    
    if description and description[-3:] != '...':
        description = f"\n\nDescription: {description}"
    else:
        description = ''
    
    if '?' in clickbait:
        clickbait = f"\n\nQuestion: {clickbait}\n\nAnswer:"
    elif [x for x in words if x in clickbait.lower()]:
        clickbait = f"\n\n{clickbait}:"
    else:
        clickbait = f"\n\nStatement to be answered: {clickbait}\n\nAnswer:"
    
    
    prompt = f"{title}{description}\n\nArticle: {article}{tags}{clickbait}"
    return prompt.strip()

print(make_prompt(validation[3]))

Article title: Alan Rickman & Rupert Grint On 'CBGB,' Reuniting Post-'Potter' And Favorite Halloween Costumes

Article: The mythology of punk music's evolution can be traced back, more or less, to one singular staple: CBGB. Opened in 1973 in downtown Manhattan's East Village, the nightclub that originally intended to house country, bluegrass and blues music (hint its acronym) quickly became a breeding ground for bands like The Ramones, Talking Heads and The Police, among countless others. The legacy of the club, which closed in 2006 over a rent dispute, is on display in a new movie that stars Alan Rickman as bemused owner Hilly Kristal. It's a bit of a departure for anyone more accustomed to seeing Rickman, 67, play sinister Severus Snape in "Harry Potter" or Judge Turpin in "Sweeney Todd." The movie also afforded him the opportunity to reunite with fellow Hogwarts alum Rupert Grint, who portrays one-third of derriere-bearing punk band The Dead Boys, alongside Justin Bartha and Bronson

In [42]:
prompt = make_prompt(validation[50])
inputs = tokenizer(prompt, return_tensors="pt") #??

NameError: name 'tokenizer' is not defined

In [ ]:
pred = tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=result_length
                      )[0])

In [ ]:
# Beam Search
#print(tokenizer.decode(model.generate(inputs["input_ids"],
#                       max_new_tokens=result_length, 
#                       num_beams=2, 
#                       no_repeat_ngram_size=2,
#                       early_stopping=True
#                      )[0]))

In [15]:
API_URL = "https://api-inference.huggingface.co/models/bigscience/bloom"
HF_TOKEN = 'TOKEN GOES HERE'
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

import time
def text_generate(prompt):
    p = prompt
    json_ = {"inputs": p,
            "parameters":
            {
          "max_new_tokens": 20,
          "return_full_text": True,
                "do_sample": False
          }, "options": 
              {
              "use_cache": True,
              "wait_for_model":True
              },}
    response = requests.post(API_URL, headers=headers, json=json_)
    try:
        output = response.json()
    except:
        return ""
    if output == {'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}:
        print("Reached hourly limit")
        for i in range(25):
            print(i/25)
            time.sleep(60*60)
        return text_generate(prompt)
    output_tmp = output[0]['generated_text']
    solution = output_tmp.split("\nQ:")[0]
    return solution

In [16]:
predictions_from_prompts = []
prompts = [make_prompt(x) for x in validation[538:]]

for i, prompt in enumerate(prompts):
    print(i+538,"/",len(prompts))
    #inputs = tokenizer(prompt, return_tensors="pt")
    #prediction = tokenizer.decode(model.generate(inputs["input_ids"], 
    #                       max_length=len(inputs['input_ids'][0])+20
    #                      )[0])
    prediction = text_generate(prompt)
    anchor = prompt[-15:]
    pred = prediction[prediction.find(anchor)+15:].strip('</s>').strip()
    if '\n' in pred:
        #pred = pred.replace('\n', ' ')
        pred = [x.strip() for x in pred.split('\n') if len(x)!=0]
    predictions_from_prompts.append(pred)
    print('prediction:',pred)

538 / 262
prediction: ["Hotel staff reveal guests' most GRUESOME habits", 'A:', 'The answer is', 'Hotel staff']
539 / 262
prediction: Read On!
540 / 262
prediction: ['Jose Jose, Mexican Singer, Falls Off Stage During Anniversary Concert (VIDEO)', 'A:', 'The title']
541 / 262
prediction: The United States would have to pay $634 billion to developing nations to stop climate change. The U
542 / 262
prediction: 1. The unemployment rate in November fell to 4.6 percent, the lowest in nine year
543 / 262
prediction: How will humans look 100,000 years from now? This question was originally answered on Quora by
544 / 262
prediction: The Supreme Court has allowed a plea of a Mumbai-based 22-year-old woman to terminate her pregnancy after taking
545 / 262
prediction: No, not yet.
546 / 262
prediction: Yes, it can. The article above shows that whale ear wax can be used to determine the stre
547 / 262
prediction: The temple officials decided to replace all the larger vessels with those of smaller s

In [ ]:
with open(f"{name}.txt", 'w') as f:
    f.writelines([str(x)+'\n' for x in predictions_from_prompts])

In [12]:
prompts[238:][0]

'Description: Bollywood: Where size matters!\n\nArticle: Salman is a bigger star than me. I think Shah Rukh is a bigger star. Amitabh ji is a bigger star. When Salman enters it looks like a big star has entered. When I enter feels like a waiter has entered. No offense to waiters. But I just mean that when I enter it doesn’t feel like a star has entered. So Salman and Shah Rukh are bigger stars than I am. I think between myself, Aamir and Salman we have huge liking towards each other. I also feel they are bigger stars than me. If you ask me I will say that thing and if you ask Salman he will say same thing. I think it is out of respect.\n\nTags: biggest, entertainment, khan, who\n\nQuestion: Who’s The Biggest Khan In Bollywood? He Is\n\nAnswer:'

In [ ]:
#Fjern gentagelser fra predictions
#Fjern overskrifter fra predictions
#Fjern A: fra predictions
#Fjern Explanation: fra predictions
#Fjern Reasoning: fra predictions

In [77]:
df = pd.read_json('input.jsonl', lines=True)

In [78]:
spoilers = df[["uuid","postId"]]
spoilers['spoiler'] = clean_preds

json_output = spoilers.to_json(orient='records', lines=True)

with open(f"{name}.jsonl", 'w') as f:
    f.write(json_output)

/tmp/ipykernel_393/742884195.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spoilers['spoiler'] = clean_preds


In [ ]:
!python3 evaluation.py --input_run test_output.jsonl --task 2 --ground_truth_spoilers validation.jsonl --output_prototext output.txt

,uuid,postId,spoiler
0,2f3e30d7-972c-4812-b727-6b582de52137,420585068076101633,[The simple household item that saves lives is...
1,45425f74-4c0e-420c-aea8-6337ea91e107,4km87q,"[Gwyneth Paltrow, I think the answer is, Gwy..."
2,2fb84020-cfdb-4b7f-9cfc-b891c306c946,4s4f3b,[The new theory on Hiddleswifts romance is biz...
3,7cc8b816-a3fe-4b3f-9568-b353f87a3161,429398501613895680,"[JaVale McGee, JaVale McGee]"
4,b90a7343-ab27-4750-8e12-43f3f03fc9e5,828292353706176512,"[Alex Owens-Sarno, The title of the article i..."
...,...,...,...
995,9615b028-fa90-4bb6-9708-9d59d3cee08b,54zmeg,"[All Along the Watchtower, The song is a cove..."
996,74e4c92c-3583-4c80-b726-36f1fe63f8c9,738816406091923456,[I think the answer is that the article is a s...
997,c5ec6cc8-46b6-4ac6-a7ee-d891fb8938b7,518772848010461184,[A blood moon is coming Oct. 8 -- and heres ho...
998,a202e079-0fae-4fde-8d89-ed6be2adc17a,412608279760879616,"[Tami Erin, I think the answer is, Tami Erin..."


In [17]:
with open("spoilers BLOOM 179.txt", 'r', encoding='utf-8') as f:
    predictions = [line.strip('\n').strip('[]"\'').split(',') for line in f]

clean_preds = []
for preds in predictions:
    tmp_preds = []
    for pred in preds:
        pred = pred.replace('"','')
        pred = pred.replace("'",'')
        pred = pred.replace("\\",'')
        tmp_preds.append(pred)
    clean_preds.append(tmp_preds)


In [28]:
df = pd.read_json('validation.jsonl', lines=True)

spoilers = df[["uuid","postId"]]
spoilers['spoiler'] = clean_preds

json_output = spoilers.to_json(orient='records', lines=True)

with open(f"{name}.jsonl", 'w') as f:
    f.write(json_output)

/tmp/ipykernel_134/1262970620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spoilers['spoiler'] = clean_preds


In [21]:
len(clean_preds)

1600

In [25]:
t = []
for i in range(1,1600,2):
    t.append(clean_preds[i])

In [27]:
clean_preds = t